### Парсинг телефонов с Авито

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

import time

import os
import re

from tqdm import tqdm

# Библиотеки для парсинга 
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager

# Парсер 
from tools import get_ad_urls, get_photos
from parser import parse_avito_page

### Константы

In [2]:
URL_TEMPLATE = "https://www.avito.ru/sankt-peterburg/telefony/mobilnye_telefony/apple-ASgBAgICAkS0wA3OqzmwwQ2I_Dc?p="
PAUSE_DURATION_SECONDS = 2
NUM_PAGES = 100
columns = [
    'title', 'price', 'characteristics', 
    'description', 'views', 'date', 'rating',
    'location', 'link'
]


### Запуск сервиса

In [3]:
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service)
service = Service(GeckoDriverManager().install())
# driver = webdriver.Firefox(service=service)
# dont open browser
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
driver = webdriver.Firefox(service=service, options=options)

### Максимальный номер сохраненной страницы  

In [4]:
output_folder = "data8"
os.makedirs(output_folder, exist_ok=True)

# Получаем список всех файлов в папке
files = os.listdir(output_folder)

# Ищем файлы с расширением .csv и извлекаем числовые части
numbers = []
for file in files:
    if file.endswith('.parquet'):
        match = re.search(r'\d+', file)
        if match:
            numbers.append(int(match.group()))

# Находим максимальное число
max_number = max(numbers) if numbers else None
print(f"Максимальный номер: {max_number}")


if not max_number: 
    start_page_number = 1
else: 
    start_page_number = max_number + 1

end_page_number = start_page_number + NUM_PAGES

print(f"Начнем парсинг со страницы номер: {start_page_number} (вкл)")
print(f"Закончим парсинг на странице номер: {end_page_number} (не вкл)")

Максимальный номер: 2
Начнем парсинг со страницы номер: 3 (вкл)
Закончим парсинг на странице номер: 103 (не вкл)


### Парсинг

In [5]:
print(output_folder)

for page_num in range(start_page_number, end_page_number):
    
    print(f"Начали парсинг страницы # {page_num}")

    df_page = pd.DataFrame(columns=columns)
    
    try:
        # Загрузка страницы с объявлениями
        url = URL_TEMPLATE + str(page_num)
        driver.get(url)
        time.sleep(PAUSE_DURATION_SECONDS)

        # Получаем HTML-код страницы
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
  
        # Извлекаем ссылки на объявления на текущей странице
        links = get_ad_urls(soup)

        for link in tqdm(links):
            
            try:
                # Переход на страницу объявления
                driver.get(link)
                time.sleep(PAUSE_DURATION_SECONDS)  # Задержка для полной загрузки

                # Парсим данные на странице объявления (название, цена, фото, описание и т.д.)
                ad_data = parse_avito_page(driver=driver) # <- словарик 
                ad_data['link'] = link
                # ad_data['photo'] = get_photos(driver=driver, cnt=5)

                df_page = pd.concat([df_page, pd.DataFrame([ad_data])], ignore_index=True)

                
            except TimeoutException:
                print(f"Ошибка: объявление {link} не загрузилось, пропускаем...")
                continue
            except Exception as e:
                print(f"Произошла ошибка при обработке объявления {link}: {e}")
                continue

        # Сохраняем данные для текущей страницы в DataFrame
        df_page.to_parquet(f'{output_folder}/phones_data_page_{page_num}.parquet')

    except TimeoutException:
        print(f"Ошибка: страница {page_num} не загрузилась, пропускаем...")
        continue
    except Exception as e:
        print(f"Произошла ошибка на странице {page_num}: {e}")
        continue



data8
Начали парсинг страницы # 3


  8%|▊         | 4/50 [00:22<04:17,  5.60s/it]


KeyboardInterrupt: 

In [ ]:
driver.quit()

In [ ]:
# read data from parquet files
data = []
for file in os.listdir(output_folder):
    if file.endswith('.parquet'):
        df = pd.read_parquet(f'{output_folder}/{file}')
        data.append(df)

# get parquet files in 
data[0].head(20)


,title,price,characteristics,description,views,date,rating,location,link
0,"iPhone 16 Pro, 1 ТБ",156890,"[СостояниеНовое, Производитель:Apple, Модель:iPhone 16 Pro, Встроенная память:1024 ГБ, Цвет:Чёрный]","iPhоne 16 Pro 1ТB, Вlасk Тitаnium (чeрный)",415 просмотров,· 27 ноября в 09:53,"4,9","Санкт-Петербург, 2-я Советская ул., 7",https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_1_tb_4247536378
1,None,117400,"[Комплект:Коробка, Провод зарядки, Коробка запечатана:Да]",📱iРhonе 16 Prо 256GВ Desert Тitаnium📱,655 просмотров,· 28 ноября в 15:47,"4,9",Санкт-Петербург,https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_256_gb_4506831908
2,None,127000,"[Комплект:Коробка, Коробка запечатана:Да]","БEСПPОЦEHTHАЯ РАСCРOЧКА 6- 36 мeсяцeв✅ Тoлькоopигинaльнaя пpoдукция и честная ценa!✅При поceщeнии нашeго магaзина и пoкупкe 📱 - aкcеccуаp в пoдaрок ❗️✅ Уcлуга пo пeрeнocу данныx на новoe устpoйство !✅ Вcетoвapы в наличии, большe предлoжeний cмотopитe в профиле!➡️🎉 Современный и стильныймагазин электроники по адресу 📍Санкт-Петербург, Коломенская улица, 33/40 ❗️Приезжайте, смотрите, покупайте!Угощаем ароматным кофе ☕️Свяжитесь прямо сейчас и начните пользоваться Вашим новым устройством уже сегодня ❗️❤️ Добавляйте объявление вИЗБРАННОЕ, чтобы не потерять наш контакт!🔥🔥🔥 Новый телефон",2003 просмотра,· 21 ноября в 19:32,"5,0",Санкт-Петербург,https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_max_256_gb_4467990928
3,"iPhone 12 mini, 128 ГБ",19090,"[Экран:Без дефектов, Корпус:Без дефектов]",⚡️Приcоeдиняйтесь cкoрее к числу дoвольныx покупателeйMIRAPНОNE!⚡️⭐️ Paccрочка ⭐️ Тradе-in ⭐️ Выкуп Baшeго тeлефонa ⭐️ Доcтaвкa пo гоpoду и облacти ⭐️……………………….👋MIRАРHОNE-Mы откpыты для Baс в14 pазных гopодaxРосcии! У наc Bы найдeте большой accoртимeнт мoдeлей айфонов и аксессуаров для них 😉……………………….📱Аррlе iРhоnе 12 mini 128gb бу,3843 просмотра,· 18 ноября в 15:40,"4,8","Санкт-Петербург, Садовая ул., 38",https://www.avito.ru/sankt-peterburg/telefony/iphone_12_mini_128_gb_4106716130
4,"iPhone 16 Pro, 512 ГБ",136990,"[СостояниеНовое, Производитель:Apple, Модель:iPhone 16 Pro, Встроенная память:512 ГБ, Цвет:Золотистый]",Арplе iРhone 16 Prо 512 ГБ✅,1071 просмотр,· 22 ноября в 00:37,"5,0","Санкт-Петербург, Полтавская ул., 8",https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_512_gb_4180905452
5,"iPhone 16 Pro Max, 1 ТБ",164700,"[Комплект:Коробка, Провод зарядки, Коробка запечатана:Да]",Hовый. Зaпечaтанный. Oригинал. Корoбка зaпечатaннaя. Не активирoвaнный.,1364 просмотра,· 13 ноября в 16:09,"5,0","Санкт-Петербург, ул. Жуковского, 28",https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_max_1_tb_4311465286
6,"iPhone 16 Pro Max, 256 ГБ",129990,"[Комплект:Коробка, Провод зарядки, Коробка запечатана:Да]","❗️ НОВЫE, НЕАКТИВИРОBАHНЫЕ, c гaрaнтиeй Аpple ❗️",152 просмотра,· 26 ноября в 15:57,"4,9","Санкт-Петербург, ул. Ефимова, 3АлитД",https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_max_256_gb_4319927192
7,"iPhone 16 Pro Max, 256 ГБ",123900,"[Комплект:Коробка, Коробка запечатана:Да]",iPhоne 16 Рro Мax 256 gb Nаturаl Тitаnium NEWВaриaнт cим-кapт: еSIМ + еSIМBapиaнты c другими SIМ также в нaличии.,2317 просмотров,· 30 ноября в 10:30,"4,9",Санкт-Петербург,https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_max_256_gb_4399948650
8,"iPhone 16 Pro, 128 ГБ",105700,"[Комплект:Коробка, Провод зарядки, Коробка запечатана:Да]",Hовый. Зaпечатaнный. Oригинал. Корoбка зaпечатаннaя. Нe aктивирoвaнный.,1766 просмотров,· 13 ноября в 16:09,"5,0","Санкт-Петербург, ул. Жуковского, 28",https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_128_gb_4310930804
9,None,149990,"[Комплект:Коробка, Провод зарядки, Коробка запечатана:Да]",В наличииiРhonе 16 Pro Мах 512Gb Whitе Тitanium,63 просмотра,· 6 декабря в 21:20,"4,8",Санкт-Петербург,https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_max_512_gb_4387389760
